# Etapes

1. Fusionner les jeux de données de test et de train
2. Appliquer des pré-traitements (élimination des stop words, passage en minuscules/majuscules, lemmatisation, racinisation (stemming), tokenisation, vectorisation, NER, POS tagging, ...) et sauvegarder chacun des jeux de données traités pour gagner du temps (comme expliqué par M. Poncelet en TD)
3. Ingénierie des données : choisir une méthode de représentation du texte (par exemple bag of words ou tf-idf) de manière à ce qu'il soit compréhensible et utilisable pour un modèle de classification, sortir des features. Potentiellement : modélisation par sujets (topic modeling), reconnaissance d'entités (entity recognition).
4. Choisir un classifieur adapté. Nous nous intéresserons à trois tâches de classification :

    - {VRAI} vs. {FAUX} (deux classes)
    - {VRAI ou FAUX} vs. {AUTRE} (deux classes)
    - {VRAI} vs. {FAUX} vs. {MIXTE} vs. {AUTRE} (quatre classes)

    Appliquer de l'upsampling ou du downsampling pour équilibrer les données.
    
    Modèles de classification envisagés : arbres de décision, SVMs, Naïve Bayes, les K-NN, les random forests, etc.
5. Sélection de variable (feature selection): pour chacune des trois tâches de classification, en plus des modèles de classification, préparer une liste de features discriminantes en ordre décroissant. Pour cela, s'appuyer sur des méthodes de sélection de variables (ou de features). Tirer les conclusions.
6. Analyse des erreurs, validation et comparaisons des modèles : comparer empiriquement les différents choix quevous avez pu faire dans la partie sélection des features, des prétraitements, des modèles
utilisés, de l'échantillonnage, etc. par rapport à leur impact sur la qualité de la classification.

## Import des dépendances nécessaires au projet

In [104]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

[nltk_data] Downloading package stopwords to /Users/Louis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Louis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Louis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Louis/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Première étape : fusionner les jeux de données de test et de train

In [105]:
train_dataframe = pd.read_csv("../data/HAI817_Projet_train.csv")
test_dataframe = pd.read_csv("../data/HAI817_Projet_test.csv")
print("Nb lignes train : " + str(len(train_dataframe)))
print("Nb lignes test : " + str(len(test_dataframe)))
dataframe = pd.concat([train_dataframe,test_dataframe])
print("Nb lignes combiné : " + str(len(dataframe)))
dataframe.to_csv("../data/combined_dataframe.csv")
dataframe = pd.read_csv("../data/combined_dataframe.csv")

Nb lignes train : 1264
Nb lignes test : 612
Nb lignes combiné : 1876


## Deuxième étape : appliquer des prétraitements sur le jeu de données

#### Ici, on écrit les fonctions de prétraitement

In [106]:
# Mettre en minuscule
def to_lower_case(text):
    if pd.isna(text):
        return text
    return text.lower()

# Mettre en majuscule
def to_upper_case(text):
    if pd.isna(text):
        return text
    return text.upper()

# Générer un wordcloud
def generate_wordcloud(data):
    text = ' '.join(data)
    wordcloud = WordCloud(width = 800, height = 800, 
                          background_color ='white', 
                          stopwords = set(stopwords.words('english')), 
                          min_font_size = 10).generate(text)
    plt.figure(figsize = (8, 8), facecolor = None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)
    plt.show()

# Suppression des caractères spéciaux, des chiffres, des stopwords et lemmatisation
def basic_preprocessing(text):
    if pd.isna(text):
        return text
    # Supprimer les caractères non-alphabétiques et les chiffres
    text = re.sub(r"[^a-zA-Z\s]", "", text, re.I|re.A)
    # Suppression des stopwords
    stop_words = set(stopwords.words("english"))
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w) for w in filtered_tokens]
    return ' '.join(lemma_words)

# Stemming
def stemming(text):
    if pd.isna(text):
        return text
    stemmer = PorterStemmer()
    word_tokens = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in word_tokens]
    return ' '.join(stemmed_words)

# POS tagging
def pos_tagging(text):
    if pd.isna(text):
        return text
    word_tokens = word_tokenize(text)
    tagged_words = nltk.pos_tag(word_tokens)
    return tagged_words

#### On créé les nouveaux datasets prétraités

**Les nouveaux datasets sont créés à partir du dataset combiné obtenu à l'étape 1**

À chaque fois, on ne traite que le texte et le titre des articles

In [ ]:
# Dataset en minuscules
lower_case_dataframe = dataframe.copy()
lower_case_dataframe["text"] = lower_case_dataframe["text"].apply(to_lower_case)
lower_case_dataframe["title"] = lower_case_dataframe["title"].apply(to_lower_case)
lower_case_dataframe.to_csv("../data/lower.csv", index=False)

# Dataset en majuscules
upper_case_dataframe = dataframe.copy()
upper_case_dataframe["text"] = upper_case_dataframe["text"].apply(to_upper_case)
upper_case_dataframe["title"] = upper_case_dataframe["title"].apply(to_upper_case)
upper_case_dataframe.to_csv("../data/upper.csv", index=False)

# Dataset avec suppression des caractères spéciaux, des chiffres, des stopwords et lemmatisation
basic_preprocessed_dataframe = dataframe.copy()
basic_preprocessed_dataframe["text"] = basic_preprocessed_dataframe["text"].apply(basic_preprocessing)
basic_preprocessed_dataframe["title"] = basic_preprocessed_dataframe["title"].apply(basic_preprocessing)
basic_preprocessed_dataframe.to_csv("../data/basic_preprocessing.csv", index=False)

# Dataset avec stemming
stemmed_dataframe = dataframe.copy()
stemmed_dataframe["text"] = stemmed_dataframe["text"].apply(stemming)
stemmed_dataframe["title"] = stemmed_dataframe["title"].apply(stemming)
stemmed_dataframe.to_csv("../data/stemming.csv", index=False)

# Dataset avec POS tagging
pos_tagged_dataframe = dataframe.copy()
pos_tagged_dataframe["text"] = pos_tagged_dataframe["text"].apply(pos_tagging)
pos_tagged_dataframe["title"] = pos_tagged_dataframe["title"].apply(pos_tagging)
pos_tagged_dataframe.to_csv("../data/pos_tagging.csv")

## Troisième étape : ingénierie des données

In [103]:
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(basic_preprocessed_dataframe)
print(X_bow)

  (0, 6)	1
  (1, 2)	1
  (2, 4)	1
  (3, 5)	1
  (4, 1)	1
  (4, 3)	1
  (5, 0)	1
